# Zach's initial analysis

Work in progress

In [1]:
from pathlib import Path
from pydantic import BaseModel
import json
from typing import List, Dict

class ObjectLabel(BaseModel):
    name: str
    bounding_box: List[int]
    area: int

class ImageAnnotation(BaseModel):
    file_path: str
    object_labels: List[ObjectLabel]
    image_area: int

# Define paths
PROJECT_ROOT = Path('.').resolve().parent
PREPROCESSED_JSON_DIR = PROJECT_ROOT / "data" / "preprocessed_jsons"
PATH_TO_TRAIN_DATA = PREPROCESSED_JSON_DIR / "train.json"
PATH_TO_TEST_DATA = PREPROCESSED_JSON_DIR / "test.json"

# Load data
with open(PATH_TO_TRAIN_DATA) as f:
    train_data = [ImageAnnotation(**item) for item in json.load(f)]
    
with open(PATH_TO_TEST_DATA) as f:
    test_data = [ImageAnnotation(**item) for item in json.load(f)]


In [2]:
train_data[0].model_dump()

{'file_path': '2011_002212',
 'object_labels': [{'name': 'person',
   'bounding_box': [200, 75, 387, 343],
   'area': 50116},
  {'name': 'person', 'bounding_box': [71, 38, 359, 344], 'area': 88128}],
 'image_area': 187500}

In [3]:
import pandas as pd

# Create list to store all object labels
all_objects = []

# Process train data
for image in train_data:
    for obj in image.object_labels:
        all_objects.append({
            'file_path': image.file_path,
            'label_name': obj.name,
            'bbox_left': obj.bounding_box[0],
            'bbox_top': obj.bounding_box[1], 
            'bbox_right': obj.bounding_box[2],
            'bbox_bottom': obj.bounding_box[3],
            'object_area': obj.area,
            'image_area': image.image_area,
            'dataset': 'train'
        })

# Process test data        
for image in test_data:
    for obj in image.object_labels:
        all_objects.append({
            'file_path': image.file_path,
            'label_name': obj.name,
            'bbox_left': obj.bounding_box[0],
            'bbox_top': obj.bounding_box[1],
            'bbox_right': obj.bounding_box[2], 
            'bbox_bottom': obj.bounding_box[3],
            'object_area': obj.area,
            'image_area': image.image_area,
            'dataset': 'test'
        })

# Create dataframe
df_objects = pd.DataFrame(all_objects)

# Display first few rows and basic info
print("DataFrame Info:")
print(df_objects.info())
print("\nFirst few rows:")
display(df_objects.head())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33256 entries, 0 to 33255
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   file_path    33256 non-null  object
 1   label_name   33256 non-null  object
 2   bbox_left    33256 non-null  int64 
 3   bbox_top     33256 non-null  int64 
 4   bbox_right   33256 non-null  int64 
 5   bbox_bottom  33256 non-null  int64 
 6   object_area  33256 non-null  int64 
 7   image_area   33256 non-null  int64 
 8   dataset      33256 non-null  object
dtypes: int64(6), object(3)
memory usage: 2.3+ MB
None

First few rows:


,file_path,label_name,bbox_left,bbox_top,bbox_right,bbox_bottom,object_area,image_area,dataset
0,2011_002212,person,200,75,387,343,50116,187500,train
1,2011_002212,person,71,38,359,344,88128,187500,train
2,2010_002413,bus,380,127,500,224,11640,178500,train
3,2010_002413,bus,273,137,339,179,2772,178500,train
4,2010_002413,bus,106,96,320,250,32956,178500,train
